# Create a Network from GeoDataFrame

<a href=https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.convert.graph_from_gdfs>`ox.graph_from_gdfs`</a> is a function in the OSMnx library that allows you to create a graph from two GeoDataFrames: one for nodes and one for edges. This function is useful when you have spatial data in the form of GeoDataFrames and want to convert it into a graph structure for analysis or visualization.
<br>

osmnx.convert.graph_from_gdfs(gdf_nodes, gdf_edges, *, graph_attrs=None)

To use this function, you need to match the nodes and edges in the GeoDataFrames with the graph structure. The nodes should be uniquely indexed by their IDs, and the edges should be uniquely multi-indexed by a combination of two nodes (u, v) and an optional key.
* gdf_nodes (GeoDataFrame) – GeoDataFrame of graph nodes uniquely indexed by osmid.
* gdf_edges (GeoDataFrame) – GeoDataFrame of graph edges uniquely multi-indexed by (u, v, key).

In [ ]:
# Import necessary libraries
import geopandas as gpd
import pandas as pd
import networkx as nx
import osmnx as ox
from tqdm import tqdm

## 1. Create a geometry of a network

In [ ]:
# Load data from ViewT (https://viewt.ktdb.go.kr/))
nodes_viewt = gpd.read_file('./data/seoul_level6_2022/seoul_node_lev6_2022.shp', encoding='cp949')
edges_viewt = gpd.read_file('./data/seoul_level6_2022/seoul_link_lev6_2022.shp', encoding='cp949')

In [ ]:
emd_gdf = gpd.read_file('./data/emd_5179.geojson')
emd_gdf['ADM_CD'] = emd_gdf['ADM_CD'].astype(str)
emd_gdf = emd_gdf.loc[emd_gdf['ADM_CD'].str.startswith('11240')]
emd_gdf = emd_gdf.dissolve(by='BASE_DATE')
emd_gdf

In [ ]:
# Select only the nodes and edges that intersect with the EMD geometry
# This is for a demonstration purpose
nodes_viewt = nodes_viewt.loc[nodes_viewt['geometry'].intersects(emd_gdf['geometry'].values[0])]
edges_viewt = edges_viewt.loc[edges_viewt['geometry'].intersects(emd_gdf['geometry'].values[0])]

## 2. Match the nodes with the graph structure

In [ ]:
# Original data
nodes_viewt.head(3)

In [ ]:
# Create a node gdf to be fed into OSMnx
nodes_gdf = nodes_viewt.copy()

nodes_gdf['node_id'] = nodes_gdf['node_id'].astype(int) # Remove decimal point
nodes_gdf['node_id'] = nodes_gdf['node_id'].astype(str) # Change to string

nodes_gdf['node_type'] = nodes_gdf['node_type'].astype(int) # Remove decimal point
nodes_gdf['node_type'] = nodes_gdf['node_type'].astype(str) # Change to string

# Assign the coordinates as columns
nodes_gdf['y'] = nodes_gdf['geometry'].y
nodes_gdf['x'] = nodes_gdf['geometry'].x

nodes_gdf = nodes_gdf[['node_id', 'node_type', 'geometry', 'x', 'y']] # Remove unnecessary columns

# Match Scheme with OSMnx
nodes_gdf = nodes_gdf.set_index('node_id')

nodes_gdf.head(3)

## 3. Match the edges with the graph structure

In [ ]:
# Original data
edges_viewt.head(3)

In [ ]:
# Original data
edges_viewt.columns

In [ ]:
# To become a MultiDiGraph, the edges should be duplicated for two-way roads
# and the geometry should be reversed for the reverse direction
# If the road is one-way, the geometry should be kept as is

edges_gdf = gpd.GeoDataFrame()

for idx, row in tqdm(edges_viewt.iterrows(), total=edges_viewt.shape[0]):
    if row['oneway'] == 0: # If the road is two-way, duplicate the row with reversed geometry
        # Create a new row with non-reversed geometry
        edges_gdf = pd.concat([gpd.GeoDataFrame({'link_id': row['up_v_link'], 
                                            'f_node': row['up_f_node'], 
                                            't_node': row['up_t_node'], 
                                            'max_speed': row['max_speed'],
                                            'road_name': row['road_name'],
                                            'road_rank': row['road_rank'],
                                            'link_type': row['link_type'],
                                            'geometry': row['geometry']
                                            }, index=[0]), edges_gdf])
        
        # Add the reverse direction
        edges_gdf = pd.concat([gpd.GeoDataFrame({'link_id': row['dw_v_link'], 
                                            'f_node': row['dw_f_node'], 
                                            't_node': row['dw_t_node'], 
                                            'max_speed': row['max_speed'],
                                            'road_name': row['road_name'],
                                            'road_rank': row['road_rank'],
                                            'link_type': row['link_type'],
                                            'geometry': row['geometry'].reverse()
                                            }, index=[0]), edges_gdf])
    else:
        # If the road is one-way, keep the row as is
        edges_gdf = pd.concat([gpd.GeoDataFrame({'link_id': row['up_v_link'], 
                                            'f_node': row['up_f_node'], 
                                            't_node': row['up_t_node'], 
                                            'max_speed': row['max_speed'],
                                            'road_name': row['road_name'],
                                            'road_rank': row['road_rank'],
                                            'link_type': row['link_type'],
                                            'geometry': row['geometry']
                                            }, index=[0]), edges_gdf])

# Reset the index
edges_gdf = edges_gdf.reset_index(drop=True)

# Assign the same CRS 
edges_gdf = edges_gdf.set_crs(edges_viewt.crs) 
edges_gdf


In [ ]:
edges_gdf['length'] = edges_gdf.apply(lambda x: x['geometry'].length, axis=1) # Calculate length in meters

# Change the data type of columns
edges_gdf['link_id'] = edges_gdf['link_id'].astype(int) # Remove decimal point
edges_gdf['link_id'] = edges_gdf['link_id'].astype(str) # Change to string

edges_gdf['f_node'] = edges_gdf['f_node'].astype(int) # Remove decimal point
edges_gdf['f_node'] = edges_gdf['f_node'].astype(str) # Change to string

edges_gdf['t_node'] = edges_gdf['t_node'].astype(int) # Remove decimal point
edges_gdf['t_node'] = edges_gdf['t_node'].astype(str) # Change to string

edges_gdf['road_rank'] = edges_gdf['road_rank'].astype(int) # Remove decimal point
edges_gdf['road_rank'] = edges_gdf['road_rank'].astype(str) # Change to string

edges_gdf['link_type'] = edges_gdf['link_type'].astype(int) # Remove decimal point
edges_gdf['link_type'] = edges_gdf['link_type'].astype(str) # Change to string

edges_gdf['max_speed'] = edges_gdf['max_speed'].astype(int) # Remove decimal point

edges_gdf

## 4. Assign information (travel speed and travel time) to the edges

In [ ]:
# Load speed data, also from ViewT
speed_df = pd.read_csv('./data/PercentileSpeed_202209.csv')
speed_df.head(3)

In [ ]:
# Select only the necessary columns
speed_df = speed_df[['﻿level6 LINK ID',  '30% 주행속도 (km/h)']]
speed_df = speed_df.rename(columns={'﻿level6 LINK ID': 'link_id', '30% 주행속도 (km/h)': 'speed'})
                                    # '평균속도 (km/h)': 'avg_speed'})
speed_df['link_id'] = speed_df['link_id'].astype(str)
speed_df

In [ ]:
# Merge the speed data with the edges data
edges_gdf = edges_gdf.merge(speed_df, on='link_id', how='left')
edges_gdf

In [ ]:
# Find the cells with missing speed data
edges_gdf.loc[(edges_gdf['speed'].isna()) | (edges_gdf['speed'] == 0)]

In [ ]:
# Fill the missing speed data based on the road rank
for idx, row in edges_gdf.loc[(edges_gdf['speed'].isna()) | (edges_gdf['speed'] == 0)].iterrows():

    if row['road_rank'] == '101': # 고속도로
        edges_gdf.at[idx, 'speed'] = 100
    elif row['road_rank'] == '102': # 도시고속도로
        edges_gdf.at[idx, 'speed'] = 80
    # 일반국도, 특별/광역시도, 국가지원지방도, 일반지방도, 시군도
    elif row['road_rank'] in ['103', '104', '105', '106', '107']: 
        edges_gdf.at[idx, 'speed'] = 50
    elif row['road_rank'] == '108': # 고속도로 연결램프
        edges_gdf.at[idx, 'speed'] = 40
    else:
        raise ValueError('Unexpected Road Rank')
            

In [ ]:
edges_gdf.loc[edges_gdf['speed'].isna()]

In [ ]:
# Match Scheme with OSMnx
edges_gdf = edges_gdf.rename(columns={'f_node': 'u', 't_node': 'v'}) #  , 'link_id': 'osmid'})
edges_gdf['key'] = 0
edges_gdf = edges_gdf.set_index(['u', 'v', 'key'])
edges_gdf

In [ ]:
edges_gdf.index.get_level_values('u').isin(nodes_gdf.index)

In [ ]:
# Remove edges that are not connected to the nodes
edges_gdf = edges_gdf.loc[(edges_gdf.index.get_level_values('u').isin(nodes_gdf.index)) & 
                          (edges_gdf.index.get_level_values('v').isin(nodes_gdf.index))
                         ]
edges_gdf

In [ ]:
# Double check if there are any edges with speed 0
edges_gdf.loc[edges_gdf['speed'] == 0]

In [ ]:
# Calculate the travel time for each edge
# travel_time = length / speed
# Convert speed from km/h to m/s
edges_gdf['travel_time'] = edges_gdf.apply(lambda x: x['length'] / (x['speed'] * 1000 / 3600), axis=1)
edges_gdf

In [ ]:
edges_gdf.plot(column='speed', cmap='Reds_r', 
               legend=True, 
               scheme='UserDefined',
                classification_kwds={'bins': [30, 50, 80, 100]},
                figsize=(10, 10))

## 5. Convert the GeoDataFrames to a graph

In [ ]:
# Convert nodes and edges GeoDataFrames to NetworkX graph
G = ox.graph_from_gdfs(nodes_gdf, edges_gdf)

ox.plot_graph(G)

## 6. Check the graph (i.e., remove disconnected nodes)

We can remove disconnected nodes from the graph using nx.strongly_connected_components() function. This function returns a list of strongly connected components in the graph, which can identify a subgraph that is unreachable from other nodes/vertices of a graph or subgraph. 

In [ ]:
list(nx.strongly_connected_components(G))

In [ ]:
abc = [c for c in sorted(nx.strongly_connected_components(G), key=len, reverse=True)]
abc

In [ ]:
nodes_gdf.loc[nodes_gdf.index.isin(abc[4])].explore()

In [ ]:
# With max function, we can find the largest connected component
largest_section = max(nx.strongly_connected_components(G), key=len)
largest_section

In [ ]:
# G.subgraph will clip the graph with the nodes provided
# This will create a new graph with only the nodes in the largest section
G = G.subgraph(largest_section)
ox.plot_graph(G)

## 7. Export the graph to a graphml file

In [ ]:
# You can save the graph to a file
# ox.save_graphml(G, './data/road_network_seoul.graphml')